In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
with open('analysis_dataset.json', 'r') as js:
     games = json.load(js)

In [3]:
gameId = '4733146699'
games[gameId]

{'win': {'100': 'Win', '200': 'Fail'},
 'ban': {'100': [89, 104, 126, 91, 360], '200': [142, 25, 117, 80, 91]},
 'pick': {'100': [76, 497, 21, 54, 777], '200': [60, 236, 29, 40, 114]},
 'pick_role': {'100': {'JUNGLE_NONE': 76,
   'BOTTOM_DUO_SUPPORT': 497,
   'BOTTOM_DUO_CARRY': 21,
   'TOP_SOLO': 54,
   'MIDDLE_SOLO': 777},
  '200': {'JUNGLE_NONE': 60,
   'MIDDLE_SOLO': 236,
   'BOTTOM_DUO_CARRY': 29,
   'BOTTOM_DUO_SUPPORT': 40,
   'TOP_SOLO': 114}}}

In [4]:
df_champion = pd.read_csv('../../data/csv/champion.csv')
df_champion = df_champion.drop(['Unnamed: 0'], axis=1)
column = ['champion','championId']
df_champion.tail()

,id,key,line
147,Zed,238,2
148,Ziggs,115,2
149,Zilean,26,4
150,Zoe,142,2
151,Zyra,143,4


In [5]:
champion_id_list = list(df_champion.key)
champion_id_list.sort()

# Prior probability( = P(H))

In [6]:
def safe_divide(x, y):
    """
    prevent from dividing by 0
    :param x: numerator
    :param y: denominator
    :return: float
    """
    try:
        return round(x / y, 4)
    except ZeroDivisionError:
        return 0

# Champion win rate( = P(H))

In [7]:
def champion_win_rate(champion_id):
    """
    챔피언 하나의 승률.
    :param champion_id: championId
    :return: [승률, 챔피언이 플레이한 게임수, 챔피언이 플레이해서 이긴 게임수]
    """
    game_count = 0
    win_count = 0
    teamIds = ['100', '200']
    for gameId in games.keys():
        for teamId in teamIds:
            if champion_id in games[gameId]['pick'][teamId]:
                game_count += 1
                if games[gameId]['win'][teamId] == 'Win':
                    win_count += 1
    return [safe_divide(win_count, game_count), game_count, win_count]

In [8]:
champion_win_rate(498)

[0.5397, 63, 34]

In [9]:
champion_win_rates = pd.DataFrame()
for champion_id in champion_id_list:
    champion_win_rates = champion_win_rates.append([[champion_id, 
                                                     champion_win_rate(champion_id)[0],
                                                     champion_win_rate(champion_id)[1],
                                                     champion_win_rate(champion_id)[2]]])
champion_win_rates.columns = ['champion_id', 'champion_win_rate', 'game_count', 'champion_win_count']
champion_win_rates.reset_index(inplace=True)
del champion_win_rates['index']

In [10]:
champion_win_rates.sort_values('champion_win_rate', ascending=False)

,champion_id,champion_win_rate,game_count,champion_win_count
137,420,1.0000,3,3
44,45,0.7500,4,3
33,34,0.7143,7,5
84,101,0.7000,10,7
51,56,0.6290,186,117
...,...,...,...,...
32,33,0.2500,12,3
35,36,0.2500,8,2
36,37,0.1429,7,1
16,17,0.0000,2,0


# Evidence( = P(E))
## P(E) = P(E|H) P(H) + P(E|~ H) P(~ H)

